## Enjoy! 

In [1]:
from os import listdir
from os.path import isfile, join
mypath = 'txt/'

onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [2]:
names = open('names.txt', 'w')
for name in onlyfiles:
    names.write(name + '\n')
names.close()

In [3]:
from collections import defaultdict

ind = defaultdict()

for i in range(len(onlyfiles)):
    ind[onlyfiles[i]] = i
    
files = []
for path in onlyfiles:
    file = open(mypath + path, 'r').read()
    files.append(file)
    
words = []
for file in files:
    words.append(file.split(' '))
    
all_words = []
for w in words:
    all_words.extend(w)
all_words_set = list(set(all_words))

ind_word = defaultdict()
for i in range(len(all_words_set)):
    ind_word[all_words_set[i]] = i

In [4]:
import numpy as np
A = np.zeros((len(all_words_set), len(files)))
for i in range(len(words)):
    for j in range(len(words[i])):
        A[ind_word[words[i][j]], i] += 1
        
full_counts = np.sum(A, axis=1)
P = (A.T / full_counts).T

P[P==0] = 1e-100

log_P = np.log(P)
log_P = log_P / np.log(len(words))
g = np.sum(P * log_P, axis = 1)
A_norm = (np.log(A+1).T * g).T

In [5]:
from scipy.linalg import svd

T, s, D = svd(A_norm, full_matrices=False)

In [37]:
r = 100
T_my = T[:, :r]
s_my = s[:r]

In [6]:
r = 300
T_my = T[:, :r]
s_my = s[:r]

In [7]:
s_my = 1 / s_my

In [8]:
trans = T_my.dot(np.diag(s_my))

In [9]:
vects = A_norm.T.dot(trans)

In [10]:
result = np.zeros((len(words), len(words)))

In [11]:
for i in range(len(words)):
    for j in range(len(words)):
        result[i, j] = np.sum(vects[i,:] * vects[j,:]) / \
        np.sqrt(np.sum(vects[i,:]* vects[i,:]) * np.sum(vects[j,:] * vects[j,:]))

In [12]:
file = open('table.tsv', 'w')
for i in range(len(words)):
    for j in range(len(words)):
        if j != len(words)-1:
            file.write(str(result[i, j]) + '\t')
        else:
            file.write(str(result[i, j]))
    file.write('\n')
file.close()

In [13]:
import pandas as pd
matr = pd.read_csv('table.tsv', header=None, sep='\t')

In [14]:
matr

,0,1,2,3,4,5,6,7,8,9,...,393,394,395,396,397,398,399,400,401,402
0,1.000000,-0.035120,-0.021573,-0.021688,-0.002877,-0.025841,-0.015358,-0.048324,-0.020364,-0.006140,...,0.018370,0.055452,-0.010216,-0.034621,0.018987,-0.004303,-0.050644,0.028139,-0.004304,-0.021913
1,-0.035120,1.000000,0.001283,0.024560,0.025791,-0.018189,0.030761,0.013319,0.000973,-0.095380,...,0.023981,0.015718,-0.012515,0.002100,-0.033989,-0.021183,0.049651,0.016191,0.033144,-0.044112
2,-0.021573,0.001283,1.000000,-0.038200,-0.042851,-0.023257,0.037100,0.033997,0.004973,0.025306,...,-0.007089,-0.037394,0.036738,0.006689,0.007783,-0.001627,-0.016711,0.019890,-0.010657,0.025390
3,-0.021688,0.024560,-0.038200,1.000000,0.005998,0.027385,-0.011539,0.034943,0.038350,0.124206,...,-0.050603,0.008041,-0.026362,0.155830,0.022698,-0.026113,0.040751,0.011384,-0.023638,0.066778
4,-0.002877,0.025791,-0.042851,0.005998,1.000000,0.001202,-0.056249,-0.069985,0.009017,0.000369,...,-0.019078,-0.037917,-0.015801,-0.038609,0.045952,-0.004782,-0.012881,-0.034477,-0.013523,-0.032284
5,-0.025841,-0.018189,-0.023257,0.027385,0.001202,1.000000,0.025356,0.046238,-0.001131,-0.000927,...,0.011963,0.016332,-0.020060,-0.044062,-0.000752,-0.004219,-0.027949,-0.004203,-0.009213,0.019488
6,-0.015358,0.030761,0.037100,-0.011539,-0.056249,0.025356,1.000000,-0.053917,-0.004820,0.069498,...,0.018920,-0.012165,0.016626,0.043756,0.014020,-0.003350,0.026839,0.022794,-0.011226,-0.049045
7,-0.048324,0.013319,0.033997,0.034943,-0.069985,0.046238,-0.053917,1.000000,0.016963,-0.022694,...,-0.036136,0.023864,-0.040621,0.027863,0.013223,0.028401,0.045466,0.030232,0.004557,-0.027141
8,-0.020364,0.000973,0.004973,0.038350,0.009017,-0.001131,-0.004820,0.016963,1.000000,-0.007033,...,-0.013884,-0.011946,0.007337,-0.045089,-0.031299,0.012976,-0.046285,0.022811,-0.010563,-0.018960
9,-0.006140,-0.095380,0.025306,0.124206,0.000369,-0.000927,0.069498,-0.022694,-0.007033,1.000000,...,0.023728,-0.026719,-0.060398,0.003040,-0.038958,-0.014092,-0.050511,-0.049410,0.005677,-0.061906


In [15]:
top_ind = (result * -1).argsort(axis = 1)

In [16]:
tops = open('tops2.txt', 'w')
for i in range(len(onlyfiles)):
    tops.write('file number: ' + onlyfiles[i][:4] + '; ')
    tops.write('top-5 numbers: ')
    for j in top_ind[i,1:6]:
        tops.write(onlyfiles[j][:4] + ' ')
    tops.write('\n')
tops.close()

In [17]:
print('file:')
print(onlyfiles[2])
top_zer = top_ind[2, 1:6]
print('top-5:')
for i in top_zer:
    print(onlyfiles[i])

file:
5814-parallel-correlation-clustering-on-big-graphs.txt
top-5:
5881-optimization-monte-carlo-efficient-and-embarrassingly-parallel-likelihood-free-inference.txt
5648-learning-to-transduce-with-unbounded-memory.txt
5909-learning-causal-graphs-with-small-interventions.txt
5969-sparse-local-embeddings-for-extreme-multi-label-classification.txt
5699-gap-safe-screening-rules-for-sparse-multi-task-and-multi-class-models.txt
